**1. I start with importing the necessary libraries**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

**2. Load the dataset**

In [2]:
df = pd.read_csv("Breast_cancer_data.csv")

**3. Lets consider X to be the feature and Y to be the target variable and diagonosis is the target in this case**

In [3]:
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

**4. Before I split the training and testing sets, I shall first encode the target variable**

In [5]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

**5. Now I can move on to splitting the dataset into training and testing sets, I shall do a 75-25 split**

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.25, random_state=42
)

**6. As per the tutorial, I also normalized the data since it is helpful in SVM problems**

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

**7. Starting with the grid search, first I assigned values to search for the best regularization parameter. For this case I set the estimator as SVC and kernel is set to linear since it is a linear SVM problem and I went with 5 fold cross validation and retrieved the best accuracy it could calculate. As we can see grid search found that 100 was the best hyperparameter and it is 93% accurate**

In [9]:
param_grid = {
    'C': [0.1, 1, 10, 100, 1000]
}

grid_search = GridSearchCV(
    estimator=SVC(kernel='linear'),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print("Best parameters found by Grid Search:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

best_svc = grid_search.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best parameters found by Grid Search: {'C': 100}
Best cross-validation accuracy: 0.9318741450068402


**8. Now we have to evaluate the model on the test set, first we do the classification report on the test set and next the confusion matrix with the test set according to the tutorial**

In [10]:
y_pred = best_svc.predict(X_test_scaled)

print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix on Test Set:")
print(confusion_matrix(y_test, y_pred))


Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        54
           1       0.96      0.96      0.96        89

    accuracy                           0.94       143
   macro avg       0.94      0.94      0.94       143
weighted avg       0.94      0.94      0.94       143

Confusion Matrix on Test Set:
[[50  4]
 [ 4 85]]


**9. As per the homework requirements, I will extract the support vectors, the indicies and the count now.**

In [11]:
support_indices = best_svc.support_
print("\nIndices of Support Vectors:", support_indices)

support_vectors = best_svc.support_vectors_
print("\nSupport Vectors (feature values after scaling):\n", support_vectors)

n_support_vectors = best_svc.n_support_
print("\nNumber of Support Vectors for Each Class:", n_support_vectors)


Indices of Support Vectors: [  1   5   7  40  66  71 101 105 108 118 133 144 148 154 164 168 173 207
 254 281 295 300 303 306 323 325 331 335 336 362 374 380 406 412 416 417
 424  24  30  32  36  43  78  97 120 122 131 151 152 165 167 194 211 218
 250 252 255 276 290 297 304 305 326 327 349 351 353 359 373 376 390 391
 392 407 408]

Support Vectors (feature values after scaling):
 [[-2.04686647e-01  3.12640109e-01 -1.33672558e-01 -2.75879949e-01
   1.07807258e+00]
 [ 3.27802485e-01  7.59894168e-01  2.89792452e-01  1.56396038e-01
  -3.57322049e-01]
 [ 2.76819483e-01  6.67603648e-01  2.21411916e-01  1.62575421e-01
  -3.58038671e-01]
 [ 5.62890772e-01 -2.71866519e-01  5.44365776e-01  4.44018267e-01
   1.18515211e-01]
 [-1.45206478e-01  1.29943875e+00 -1.69922481e-01 -2.31219857e-01
  -2.13280950e-01]
 [-6.46539330e-01 -1.18048985e-01 -5.74850600e-01 -6.01982886e-01
   1.09957125e+00]
 [-4.73763602e-01 -8.27976064e-01 -3.83714641e-01 -4.99461291e-01
   2.31066307e+00]
 [ 4.21271322e-01  4

**10. Next homework requirement is to list the weight vectors which is nothing but the coefficients from the linear SVM, I found that for a linear SVM .coef returns a 2D array of classes and features. In a binary classification this would be 1 and features.**

In [12]:
weight_vector = best_svc.coef_[0]
print("\nWeight Vector (Coefficients) for Each Feature:\n", weight_vector)


Weight Vector (Coefficients) for Each Feature:
 [ 8.21548018 -1.05696857 -6.52080945 -5.39162943 -1.21269819]


In [13]:
bias = best_svc.intercept_[0]
print("\nIntercept (Bias) of the Model:\n", bias)


Intercept (Bias) of the Model:
 0.2413017455368774


**11. In order to see what decision our model arrives at, I shall pick one test sample**

In [16]:
test_sample_index = 0
test_sample = X_test_scaled[test_sample_index].reshape(1, -1)

prediction = best_svc.predict(test_sample)[0]
predicted_class = le.inverse_transform([prediction])[0]

print(f"\nTest Sample Index {test_sample_index} Predicted Class:", predicted_class)

decision_value = best_svc.decision_function(test_sample)[0]
print(f"Decision Function Value for Sample {test_sample_index}: {decision_value:.4f}")

feature_names = X.columns
contributions = weight_vector * test_sample[0]

feature_contributions = list(zip(feature_names, contributions))

feature_contributions_sorted = sorted(
    feature_contributions, key=lambda x: abs(x[1]), reverse=True
)

print("\nFeature Contributions to the Decision:")
for feat, contrib in feature_contributions_sorted:
    print(f"{feat}: {contrib:.4f}")


Test Sample Index 0 Predicted Class: 1
Decision Function Value for Sample 0: 1.7096

Feature Contributions to the Decision:
mean_radius: -3.8457
mean_perimeter: 2.8997
mean_area: 2.6202
mean_smoothness: -0.3558
mean_texture: 0.1498


# CONCLUSION:



1. Positive contributions are pushing the svm decision value towards malignant which is 1.
2. So Negative contributions are pushing the svm decision value towards benign which is 0
3. The magnitude shows us how strongly each feature influences the decision, in the final code output we can see how each feature is contributing to the decision.

